In [ ]:
# https://machinelearningmastery.com/learn-add-numbers-seq2seq-recurrent-neural-networks/

In [1]:
from random import seed
from random import randint
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


In [2]:
# Addition as a mapping problem (the beginner's mistake)

seed(1)
X, y = list(), list()
for i in range(100):
    in_pattern = [randint(1,100) for _ in range(2)]
    out_pattern = sum(in_pattern)
    print(in_pattern, out_pattern)
    X.append(in_pattern)
    y.append(out_pattern)

# format as NumPy arrays
X,y = np.array(X), np.array(y)
# normalize
X = X.astype('float') / float(100 * 2)
y = y.astype('float') / float(100 * 2)

# generate examples of random integers and their sum
def random_sum_pairs(n_examples, n_numbers, largest):
    X, y = list(), list()
    for i in range(n_examples):
        in_pattern = [randint(1,largest) for _ in range(n_numbers)]
        out_pattern = sum(in_pattern)
        X.append(in_pattern)
        y.append(out_pattern)
    # format as NumPy arrays
    X,y = np.array(X), np.array(y)
    # normalize
    X = X.astype('float') / float(largest * n_numbers)
    y = y.astype('float') / float(largest * n_numbers)
    return X, y

# invert normalization
def invert(value, n_numbers, largest):
    return round(value * float(largest * n_numbers))

# generate training data
n_examples = 100
n_numbers = 2
largest = 100
# define LSTM configuration
n_batch = 1
n_epoch = 100

# create LSTM
model = Sequential()
model.add(LSTM(6, input_shape=(n_numbers, 1), return_sequences=True))
model.add(LSTM(6))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# train LSTM
for _ in range(n_epoch):
    X, y = random_sum_pairs(n_examples, n_numbers, largest)
    X = X.reshape(n_examples, n_numbers, 1)
    model.fit(X, y, epochs=1, batch_size=n_batch, verbose=2)
    
# evaluate on some new patterns
X, y = random_sum_pairs(n_examples, n_numbers, largest)
X = X.reshape(n_examples, n_numbers, 1)
result = model.predict(X, batch_size=n_batch, verbose=0)
# calculate error
expected = [invert(x, n_numbers, largest) for x in y]
predicted = [invert(x, n_numbers, largest) for x in result[:,0]]
rmse = np.sqrt(mean_squared_error(expected, predicted))
print('RMSE: %f' % rmse)
# show some examples
for i in range(20):
    error = expected[i] - predicted[i]
    print('Expected=%d, Predicted=%d (err=%d)' % (expected[i], predicted[i], error))

[18, 73] 91
[98, 9] 107
[33, 16] 49
[64, 98] 162
[58, 61] 119
[84, 49] 133
[27, 13] 40
[63, 4] 67
[50, 56] 106
[78, 98] 176
[99, 1] 100
[90, 58] 148
[35, 93] 128
[30, 76] 106
[14, 41] 55
[4, 3] 7
[4, 84] 88
[70, 2] 72
[49, 88] 137
[28, 55] 83
[93, 4] 97
[68, 29] 97
[98, 57] 155
[64, 71] 135
[30, 45] 75
[30, 87] 117
[29, 98] 127
[59, 38] 97
[3, 54] 57
[72, 83] 155
[13, 24] 37
[81, 93] 174
[38, 16] 54
[96, 43] 139
[93, 92] 185
[65, 55] 120
[65, 86] 151
[25, 39] 64
[37, 76] 113
[64, 65] 129
[51, 76] 127
[5, 62] 67
[32, 96] 128
[52, 54] 106
[86, 23] 109
[47, 71] 118
[90, 100] 190
[87, 95] 182
[48, 12] 60
[57, 85] 142
[66, 14] 80
[100, 21] 121
[67, 51] 118
[48, 63] 111
[94, 4] 98
[61, 6] 67
[40, 91] 131
[79, 76] 155
[75, 51] 126
[83, 22] 105
[22, 65] 87
[30, 2] 32
[99, 26] 125
[70, 71] 141
[30, 52] 82
[66, 45] 111
[74, 46] 120
[59, 35] 94
[85, 71] 156
[78, 94] 172
[1, 50] 51
[95, 66] 161
[17, 67] 84
[100, 72] 172
[27, 55] 82
[8, 62] 70
[47, 73] 120
[71, 26] 97
[65, 53] 118
[63, 46] 109
[54,

In [4]:
#The problem we have solved had multiple inputs but was technically not a sequence prediction problem.
#In fact, you can just as easily solve it using a multilayer Perceptron (MLP). For example:

# generate training data
seed(1)
n_examples = 100
n_numbers = 2
largest = 100
# define LSTM configuration
n_batch = 2
n_epoch = 50

# create LSTM
model = Sequential()
model.add(Dense(4, input_dim=n_numbers))
model.add(Dense(2))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
# train LSTM
for _ in range(n_epoch):
    X, y = random_sum_pairs(n_examples, n_numbers, largest)
    model.fit(X, y, epochs=1, batch_size=n_batch, verbose=2)
# evaluate on some new patterns
X, y = random_sum_pairs(n_examples, n_numbers, largest)
result = model.predict(X, batch_size=n_batch, verbose=0)
# calculate error
expected = [invert(x, n_numbers, largest) for x in y]
predicted = [invert(x, n_numbers, largest) for x in result[:,0]]
rmse = np.sqrt(mean_squared_error(expected, predicted))
print('RMSE: %f' % rmse)
# show some examples
for i in range(20):
    error = expected[i] - predicted[i]
    print('Expected=%d, Predicted=%d (err=%d)' % (expected[i], predicted[i], error))

Epoch 1/1
 - 1s - loss: 0.2139
Epoch 1/1
 - 0s - loss: 0.0257
Epoch 1/1
 - 0s - loss: 0.0194
Epoch 1/1
 - 0s - loss: 0.0147
Epoch 1/1
 - 0s - loss: 0.0107
Epoch 1/1
 - 0s - loss: 0.0132
Epoch 1/1
 - 0s - loss: 0.0081
Epoch 1/1
 - 0s - loss: 0.0064
Epoch 1/1
 - 0s - loss: 0.0053
Epoch 1/1
 - 0s - loss: 0.0036
Epoch 1/1
 - 0s - loss: 0.0028
Epoch 1/1
 - 0s - loss: 0.0025
Epoch 1/1
 - 0s - loss: 0.0016
Epoch 1/1
 - 0s - loss: 0.0012
Epoch 1/1
 - 0s - loss: 8.9821e-04
Epoch 1/1
 - 0s - loss: 4.0829e-04
Epoch 1/1
 - 0s - loss: 3.4280e-04
Epoch 1/1
 - 0s - loss: 2.7012e-04
Epoch 1/1
 - 0s - loss: 1.9705e-04
Epoch 1/1
 - 0s - loss: 1.0518e-04
Epoch 1/1
 - 0s - loss: 6.0041e-05
Epoch 1/1
 - 0s - loss: 4.0836e-05
Epoch 1/1
 - 0s - loss: 1.7477e-05
Epoch 1/1
 - 0s - loss: 1.0031e-05
Epoch 1/1
 - 0s - loss: 5.6489e-06
Epoch 1/1
 - 0s - loss: 2.4523e-06
Epoch 1/1
 - 0s - loss: 1.8525e-06
Epoch 1/1
 - 0s - loss: 1.0718e-06
Epoch 1/1
 - 0s - loss: 4.9707e-07
Epoch 1/1
 - 0s - loss: 2.7830e-07
Epoch 

In [9]:
#We can frame addition as an input and output string of characters and let the model figure out the meaning of the characters. 
#The entire addition problem can be framed as a string of characters, such as “12+50” with the output “62”, or more specifically:
#Input: [‘1’, ‘2’, ‘+’, ‘5’, ‘0’]
#Output: [‘6’, ‘2’]

seed(1)
n_samples = 1
n_numbers = 2
largest = 10

#Padding requires we know how long the longest sequence may be. We can calculate this easily by taking the log10() of the largest 
#integer we can generate and the ceiling of that number to get an idea of how many chars are needed for each number. 
#We add 1 to the largest number to ensure we expect 3 chars instead of 2 chars for the case of a round largest number, 
#like 200. We then need to add the right number of plus symbols
# max_length = n_numbers * ceil(log10(largest+1)) + n_numbers - 1
# max_length = ceil(log10(n_numbers * (largest+1)))

# generate lists of random integers and their sum
def random_sum_pairs(n_examples, n_numbers, largest):
    X, y = list(), list()
    for i in range(n_examples):
        in_pattern = [randint(1,largest) for _ in range(n_numbers)]
        out_pattern = sum(in_pattern)
        X.append(in_pattern)
    y.append(out_pattern)
    return X, y

# convert data to strings
def to_string(X, y, n_numbers, largest):
    max_length = n_numbers * np.ceil(np.log10(largest+1)) + n_numbers - 1
    Xstr = list()
    for pattern in X:
        strp = '+'.join([str(n) for n in pattern])
        strp = ''.join([' ' for _ in range(max_length-len(strp))]) + strp
    Xstr.append(strp)
    max_length = np.ceil(np.log10(n_numbers * (largest+1)))
    ystr = list()
    for pattern in y:
        strp = str(pattern)
        strp = ''.join([' ' for _ in range(max_length-len(strp))]) + strp
        ystr.append(strp)
    return Xstr, ystr

# generate pairs
X, y = random_sum_pairs(n_samples, n_numbers, largest)
print(X, y)
# convert to strings
X, y = to_string(X, y, n_numbers, largest)
print(X, y)

[[3, 10]] [13]


TypeError: 'numpy.float64' object cannot be interpreted as an integer